In [101]:
from getpass import getpass
import warnings

from pandas.io.pytables import format_doc
from prompt_toolkit.utils import to_str

warnings.filterwarnings('ignore')
from utils import ChatOpenAI

#course_api_key= "Введите ваш API ключ с курса"
course_api_key = getpass(prompt='Введите API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

In [1]:
baza_znanii = ['хочу латте',' латте и додстер',' хочу пиццу',' хочу',' не хочу ничего',' подписывайтесь на канал в тг @big_llm_course']

doc_baza_znanii = []
for i in baza_znanii:
    doc_baza_znanii.append(str(i)) # перевожу все элементы в str т.к была ошибка(ожидание строки или битов)
 
ggg = ' ' 
for i in doc_baza_znanii:
    ggg =ggg+i    
    
    
print(ggg)
type(ggg)

 хочу латте латте и додстер хочу пиццу хочу не хочу ничего подписывайтесь на канал в тг @big_llm_course


str

In [2]:
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)

splitter = RecursiveCharacterTextSplitter(
    chunk_size=15,
    chunk_overlap=3,
    length_function=len,
)


split_documents = splitter.create_documents([ggg])
len(split_documents)
print(split_documents)

[Document(metadata={}, page_content='хочу латте'), Document(metadata={}, page_content='латте и'), Document(metadata={}, page_content='и додстер хочу'), Document(metadata={}, page_content='пиццу хочу не'), Document(metadata={}, page_content='не хочу ничего'), Document(metadata={}, page_content='подписывайтесь'), Document(metadata={}, page_content='на канал в тг'), Document(metadata={}, page_content='@big_llm_cours'), Document(metadata={}, page_content='urse')]


In [3]:
print([ggg])

[' хочу латте латте и додстер хочу пиццу хочу не хочу ничего подписывайтесь на канал в тг @big_llm_course']


In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

# Если у вас нет видеокарты, укажите 'device': 'cpu'                                          или cuda - если через видеокарту
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cpu"}
)

C:\Users\fRomanSt\AppData\Local\Temp\ipykernel_14232\2676310870.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings_model = HuggingFaceEmbeddings(
C:\Users\fRomanSt\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain.vectorstores import FAISS
from utils import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(course_api_key=course_api_key)
db = FAISS.from_documents(
    split_documents, hf_embeddings_model
)  

db.save_local("faiss_db_111") # можно сохранить базу локально, указав путь


In [6]:
db.similarity_search_with_score(
    "хочу колу и додсттер"
)

[(Document(id='68f98600-23eb-454d-866b-843ac5a9282d', metadata={}, page_content='и додстер хочу'),
  0.61294854),
 (Document(id='c447f9be-bbc4-460f-b4c0-8d90f761d893', metadata={}, page_content='хочу латте'),
  0.8483116),
 (Document(id='d31c1ff1-9d38-4d30-956d-7a7016caa27e', metadata={}, page_content='пиццу хочу не'),
  0.8882828),
 (Document(id='2e6d31a4-e2b2-4ae5-9072-4f4f689e4584', metadata={}, page_content='не хочу ничего'),
  0.9604651)]

In [7]:
                     #  1 вариант retriever(извлечение)
#  Самый частый кейс - использование векторного хранилища и его методов для получения документов
retriever = db.as_retriever(
    search_type="similarity",  # тип поиска похожих документов
    k=3,  # количество возвращаемых документов (Default: 4)
    score_threshold=None,  # минимальный порог для поиска "similarity_score_threshold"
)
# retriever.get_relevant_documents(
#     "Кого убили раздетой на крыльце?"
# )

                          #  2 вариант retriever(извлечение)

from langchain.retrievers import BM25Retriever, EnsembleRetriever

# Попробуем BM25 - алгоритм поиска по ключевым словам. Может искать лучше, но не поймёт запрос на другом языке.
# Также не сможет искать, если запрос на обывательском языке, а документы на профессиональном
bm25 = BM25Retriever.from_documents(split_documents[:70])  # Эмбеддинги ему не нужны
bm25.k = 3  # Так можно задать количество возвращаемых документов

                     #  3 вариант retriever(извлечение) Объединяем bm25 и retriever (первый и второй вариант)
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# Если один ретривер плохо справляется, то можно использовать целый ансамбль и даже каждому вес назначить
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25, retriever],  # список ретриверов
    weights=[
        0.4,
        0.6,
    ],  # веса, на которые домножается скор документа от каждого ретривера
)

retriever.get_relevant_documents(
    "хочу колу и додсттер"
)

C:\Users\fRomanSt\AppData\Local\Programs\Python\Python39\lib\site-packages\langchain\retrievers\document_compressors\chain_extract.py:13: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain.chains.llm import LLMChain
C:\Users\fRomanSt\AppData\Local\Programs\Python\Python39\lib\site-packages\pydantic\_internal\_generate_schema.py:775: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `ResponseSchema` to V2.
  warn(
C:\Users\

[Document(id='68f98600-23eb-454d-866b-843ac5a9282d', metadata={}, page_content='и додстер хочу'),
 Document(id='c447f9be-bbc4-460f-b4c0-8d90f761d893', metadata={}, page_content='хочу латте'),
 Document(id='d31c1ff1-9d38-4d30-956d-7a7016caa27e', metadata={}, page_content='пиццу хочу не'),
 Document(id='2e6d31a4-e2b2-4ae5-9072-4f4f689e4584', metadata={}, page_content='не хочу ничего')]

In [121]:
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Создаём простой шаблон         

template = """
дай ответ на вопрос только  фразами из context:

{context}

Question: {question}
"""
# Создаём промпт из шаблона
prompt = ChatPromptTemplate.from_template(template)


# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


# Создаём цепочку
chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
chain.invoke("хочу колу и додсттер")

InternalServerError: Internal Server Error